In [1]:
import numpy as np
from lammps_implicit_der.systems import Bcc, BccBinary, BccBinaryVacancy, BccVacancy, BccSIA


# Systems

In [16]:
bcc_pure = Bcc(alat=3.18427, ncell_x=2, minimize=True, logname=None, data_path='./refs/', snapcoeff_filename = 'W.snapcoeff', verbose=False)
print(bcc_pure.energy)
print(bcc_pure.Ndesc)
print(bcc_pure.dU_dTheta @ bcc_pure.Theta)

-89.060889667491
55
-89.06088966749098


In [3]:
bcc_pure = Bcc(alat=3.18, ncell_x=2, minimize=False, logname=None, data_path='./refs/', snapcoeff_filename = 'W.snapcoeff', verbose=False)
print(bcc_pure.energy)

-89.0555278168474


In [17]:
bcc_vac = BccVacancy(alat=3.163, ncell_x=2, minimize=True, logname=None, data_path='./refs/', snapcoeff_filename = 'W.snapcoeff', verbose=False)
print(bcc_vac.energy)
print(bcc_vac.Natom)

-80.36527467896518
15


In [5]:
bcc_binary = BccBinary(alat=3.13, ncell_x=2, minimize=True, logname=None, data_path='./refs/', snapcoeff_filename = 'NiMo.snapcoeff', verbose=False)
print(bcc_binary.energy)
print(bcc_binary.Natom)
print(bcc_binary.species)
print(bcc_binary.Ndesc)

-122.76839711519715
16
[1 1 1 2 2 2 2 2 2 1 1 2 1 1 1 1]
30


In [6]:
bcc_binary = BccBinaryVacancy(alat=3.13, ncell_x=2, minimize=True, logname=None, data_path='./refs/', snapcoeff_filename = 'NiMo.snapcoeff', verbose=False)
print(bcc_binary.energy)
print(bcc_binary.Natom)
print(bcc_binary.species)

-116.12471812826522
15
[1 1 1 2 2 2 2 2 2 1 1 2 1 1 1]


In [7]:
bcc_SIA = BccSIA(alat=3.18, ncell_x=2, minimize=False, logname=None, data_path='./refs/', snapcoeff_filename = 'W.snapcoeff', verbose=False)
print(bcc_SIA.energy)
print(bcc_SIA.Natom)

-53.04187280340517
17


In [8]:
alat0 = 3.175
ncell_x = 2
system = BccVacancy(alat=alat0, ncell_x=ncell_x, minimize=True, logname=None,
                    data_path='./refs/', snapcoeff_filename='W.snapcoeff', fix_box_relax=True, verbose=False)

alat = system.volume**(1/3) / ncell_x
print(alat)
print(system.energy)

3.163163264038606
-80.3652809680334


# Potential object

In [9]:
bcc_binary = BccBinary(alat=3.13, specie_B_concentration=0.2, ncell_x=2, minimize=False, logname=None, data_path='./refs/', snapcoeff_filename = 'NiMo.snapcoeff', verbose=False)
print(bcc_binary.pot.Theta_dict.keys())
print(bcc_binary.pot.Theta_dict['weights'], bcc_binary.pot.Theta_dict['radii'])
print(bcc_binary.pot.Theta_dict['Mo']['beta0'])
assert all( el in bcc_binary.pot.Theta_dict for el in ['Ni', 'Mo'])

dict_keys(['Ni', 'Mo', 'radii', 'weights'])
0.5 1.0 0.575 0.575
-11.1413071988


In [10]:
bcc_pure = Bcc(alat=3.18427, ncell_x=2, minimize=False, logname=None, data_path='./refs/', snapcoeff_filename='W.snapcoeff', verbose=False)
coeffs = np.loadtxt('./refs/W.snapcoeff', skiprows=7)
print(coeffs.shape)
print(bcc_pure.pot.Theta_dict['W']['Theta'].shape)

(55,)
(55,)


# System properties

In [19]:
bcc_vacancy = BccVacancy(alat=3.163, ncell_x=1, minimize=False, logname=None, data_path='./refs/', snapcoeff_filename='W.snapcoeff', verbose=False)
print(bcc_vacancy.energy)
print(bcc_vacancy.dU_dTheta @ bcc_vacancy.Theta)
print(bcc_vacancy.X_coord)

print(bcc_vacancy.cell)
print(bcc_vacancy.inv_cell)

-11.115719916006501
-11.1157199160065
[ 0.03163  0.03163  0.03163 -1.54987 -1.54987 -1.54987]
[[3.163 0.    0.   ]
 [0.    3.163 0.   ]
 [0.    0.    3.163]]
[[0.31615555 0.         0.        ]
 [0.         0.31615555 0.        ]
 [0.         0.         0.31615555]]


In [12]:
X_coord_test = bcc_vacancy.X_coord.copy()
X_coord_test[0] = 20.0
X_coord_test[3] = 50.0
bcc_vacancy.X_coord = X_coord_test
print(bcc_vacancy.X_coord)

[ 1.022    0.03163  0.03163 -0.608   -1.54987 -1.54987]


In [13]:
cell0 = bcc_vacancy.cell.copy()
strain = 1.5
cell_test = np.dot(cell0, np.eye(3) * strain)
bcc_vacancy.apply_strain(cell_test, update_system=True)
print(bcc_vacancy.cell)
bcc_vacancy.apply_strain(cell0, update_system=True)

[[4.7445 0.     0.    ]
 [0.     4.7445 0.    ]
 [0.     0.     4.7445]]


In [14]:
print(bcc_vacancy.volume)

31.644451746999998


In [27]:
bcc_vacancy = BccVacancy(alat=3.163, ncell_x=1, minimize=False, logname=None, data_path='./refs/', snapcoeff_filename='W.snapcoeff', verbose=False)
print(bcc_vacancy.dU_dTheta)
X_coord_test = bcc_vacancy.X_coord.copy()
X_coord_test[4] = 0.5
bcc_vacancy.scatter_coord(X_coord=X_coord_test)
bcc_vacancy.compute_D_dD()
bcc_vacancy.gather_D_dD()
print(bcc_vacancy.dU_dTheta.shape)
print(bcc_vacancy.mixed_hessian.shape)

# save mixed hessian to npy
np.save('test_system_props_dU_dTheta.npy', bcc_vacancy.dU_dTheta)
np.save('test_system_props_mixed_hessian.npy', bcc_vacancy.mixed_hessian)

# load mixed hessian from npy
mixed_hessian = np.load('test_system_props_mixed_hessian.npy')


[550.51114458   2.01479374  -5.99821035  -5.99993478  -8.00846585
  -5.99999999  -9.9999742  143.41878999 -22.66293236  -7.96996449
 -11.99406225  34.92610888  29.79360043 168.39461996 -14.73697034
  -9.97213532 -13.97448532  -7.87213139  41.26969941  30.12738214
  45.58408436  35.2644781    2.8297426  -13.76368866 -11.99952588
 -15.99261993  -4.92320288  -6.85702118 -12.4324085  -13.35457254
  -8.40087413 -17.70520886 519.47576739 -16.41943691 -14.02526168
 -17.996125    65.06167156  48.15601955  20.18407558 -22.04668129
  40.91004687  90.75393928 323.64108603 -33.01066077 -15.97312894
  12.96415603  26.55092319  -9.57943424  51.48242956  16.47068853
 300.97239624 -17.99041169  16.26521112  22.15065682   1.62815362]
(55,)
(55, 6)


In [28]:
bcc_binary = BccBinary(alat=3.13, ncell_x=1, minimize=True, logname=None,
                        data_path='./refs/', snapcoeff_filename='NiMo.snapcoeff', verbose=False)

X_coord_test = bcc_binary.X_coord.copy()
X_coord_test[4] = 0.5
bcc_binary.scatter_coord(X_coord=X_coord_test)
bcc_binary.compute_D_dD()
bcc_binary.gather_D_dD()
print(bcc_binary.dU_dTheta.shape)
print(bcc_binary.mixed_hessian.shape)

# save mixed hessian to npy
np.save('test_system_props_dU_dTheta_binary.npy', bcc_binary.dU_dTheta)
np.save('test_system_props_mixed_hessian_binary.npy', bcc_binary.mixed_hessian)

(30,)
(30, 6)


### Forces

In [41]:
bcc_vacancy = BccVacancy(alat=3.163, ncell_x=1, minimize=False, logname=None, data_path='./refs/', snapcoeff_filename='W.snapcoeff', verbose=False)
dX_vector = np.zeros(bcc_vacancy.Natom*3)
dX_vector[0] = 0.1
print(', '.join(map(str,bcc_vacancy.forces(dx=dX_vector))))

-0.05229752523716957, 9.575673587391975e-16, 2.847605337023395e-15, 0.05229752523716957, -1.0269562977782698e-15, -2.9211995299477888e-15


### Hessian

In [46]:
bcc_vacancy = BccVacancy(alat=3.163, ncell_x=1, minimize=False, logname=None, data_path='./refs/', snapcoeff_filename='W.snapcoeff', verbose=False)
print(bcc_vacancy.hessian())
np.save('test_system_props_hessian.npy', bcc_vacancy.hessian())

h_test = np.load('test_system_props_hessian.npy')
print(np.allclose(bcc_vacancy.hessian(), h_test))

[[ 1.04601487e+01  2.08166817e-13 -1.54039509e-14 -1.04581487e+01
  -1.21430643e-13  7.45542618e-14]
 [-3.05311332e-13  1.04601487e+01  2.46549801e-13  2.22044605e-13
  -1.04581487e+01 -8.52795848e-14]
 [-2.49800181e-13  1.70002901e-13  1.04601487e+01  2.77555756e-13
  -3.46944695e-13 -1.04581487e+01]
 [-1.04581487e+01  7.52869989e-13 -3.38438880e-13  1.04601487e+01
  -6.52256027e-13  3.35773745e-13]
 [ 9.43689571e-13 -1.04581487e+01 -1.23754533e-13 -7.77156117e-13
   1.04601487e+01  7.67038494e-14]
 [ 3.60822483e-13  7.63278329e-14 -1.04581487e+01 -3.33066907e-13
  -1.45716772e-13  1.04601487e+01]]
True


In [48]:
np.linspace(-0.03, 0.03, 81)

array([-0.03   , -0.02925, -0.0285 , -0.02775, -0.027  , -0.02625,
       -0.0255 , -0.02475, -0.024  , -0.02325, -0.0225 , -0.02175,
       -0.021  , -0.02025, -0.0195 , -0.01875, -0.018  , -0.01725,
       -0.0165 , -0.01575, -0.015  , -0.01425, -0.0135 , -0.01275,
       -0.012  , -0.01125, -0.0105 , -0.00975, -0.009  , -0.00825,
       -0.0075 , -0.00675, -0.006  , -0.00525, -0.0045 , -0.00375,
       -0.003  , -0.00225, -0.0015 , -0.00075,  0.     ,  0.00075,
        0.0015 ,  0.00225,  0.003  ,  0.00375,  0.0045 ,  0.00525,
        0.006  ,  0.00675,  0.0075 ,  0.00825,  0.009  ,  0.00975,
        0.0105 ,  0.01125,  0.012  ,  0.01275,  0.0135 ,  0.01425,
        0.015  ,  0.01575,  0.0165 ,  0.01725,  0.018  ,  0.01875,
        0.0195 ,  0.02025,  0.021  ,  0.02175,  0.0225 ,  0.02325,
        0.024  ,  0.02475,  0.0255 ,  0.02625,  0.027  ,  0.02775,
        0.0285 ,  0.02925,  0.03   ])